# CAPM Notebook

In [32]:
# Security market line

import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import plotly.graph_objects as go
import statsmodels.api as sm
from scipy.stats import norm

In [10]:
betas = norm.rvs(loc=1, scale=0.5, size=10,random_state=10)
mrp = 0.05
rf  = 0.02
expret = rf + betas*mrp

In [16]:
# Plot SML with no alpha
trace  = go.Scatter(x=betas, y=expret, mode="markers", name='Assets', marker=dict(size=10),)
minval = 0.0
maxval = 2.0
trace_sml = go.Scatter(x= np.linspace(minval,maxval,100), y = rf+mrp*np.linspace(minval,maxval,100), mode='lines',name='SML',
    hovertemplate="SML<br>y-intercept: risk-free rate<br>slope: market risk premium<extra></extra>")

fig = go.Figure()
fig.add_trace(trace)
fig.add_trace(trace_sml)
fig.update_xaxes(title='Beta',tickformat=".2f", range=[minval,maxval])
fig.update_yaxes(title='Expected Return',tickformat=".1%", range=[0, 0.2])
fig.update_layout(title='Security Market Line (no alpha)')
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

In [25]:
# Add alphas to some assets

alphas = np.zeros(len(betas))
alphas[0]=0.02
alphas[3]=-0.02
alphas[6]= 0.01
alphas[9]= -0.03
expret += alphas

In [27]:
# Plot SML with no alpha
trace  = go.Scatter(x=betas, y=expret, mode="markers", name='Assets', marker=dict(size=10),customdata=alphas,
    hovertemplate="<br>beta: %{x:.2f}<br>alpha: %{customdata:.1%}<extra></extra>")
minval = 0.0
maxval = 2.0
trace_sml = go.Scatter(x= np.linspace(minval,maxval,100), y = rf+mrp*np.linspace(minval,maxval,100), mode='lines',name='SML',
    hovertemplate="SML<br>y-intercept: risk-free rate<br>slope: market risk premium<extra></extra>")

fig = go.Figure()
fig.add_trace(trace)
fig.add_trace(trace_sml)
fig.update_xaxes(title='Beta',tickformat=".2f", range=[minval,maxval])
fig.update_yaxes(title='Expected Return',tickformat=".1%", range=[0, 0.2])
fig.update_layout(title='Security Market Line (with alpha)')
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

# Industries SML

In [28]:
# Read industry and clean-up missing data (coded -99.99)
ff48 = pdr("48_Industry_Portfolios", "famafrench", start=1900)[0]

# Clean-up missings
for c in ff48.columns:
    ff48[c] = np.where(ff48[c]==-99.99, np.nan, ff48[c])
ff48 = ff48/100

# Pull and merge market returns
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
df = ff48.join(ff3[['Mkt-RF','RF']])
df = df.loc['1970-01':].copy()  # There is missing data prior to 1970

In [36]:
# Estimate betas and take average return
def params(varname):
    y = df[varname]-df['RF']
    X = sm.add_constant(df['Mkt-RF'])
    mm = sm.OLS(y, X, missing='drop').fit()
    return [mm.params['Mkt-RF'], y.mean()]
params('Util ')

[0.5158115967503908, 0.005413902053712481]

In [43]:
df_beta = pd.DataFrame(index=ff48.columns, columns = ['beta','avg_ret'],dtype=float)
for c in ff48.columns:
    df_beta.loc[c,:] = params(c)
df_beta

,beta,avg_ret
Agric,0.844771,0.006379
Food,0.650510,0.006962
Soda,0.799203,0.007330
Beer,0.722205,0.007446
Smoke,0.635587,0.009831
Toys,1.178129,0.003612
Fun,1.338614,0.008741
Books,1.081942,0.004824
Hshld,0.773439,0.004763
Clths,1.114118,0.006841


In [49]:
theory_slope = df['Mkt-RF'].mean()
print(f'Under CAPM, slope of empirical CAPM should be {theory_slope: .4f} per month.')

Under CAPM, slope of empirical CAPM should be  0.0056 per month.


0.0677004739336493

In [44]:
# Empirical SML
y = df_beta.avg_ret
X = sm.add_constant(df_beta.beta)
emp_sml = sm.OLS(y, X, missing='drop').fit()
print(emp_sml.summary())

                            OLS Regression Results                            
Dep. Variable:                avg_ret   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.7473
Date:                Tue, 15 Nov 2022   Prob (F-statistic):              0.392
Time:                        14:05:58   Log-Likelihood:                 241.20
No. Observations:                  48   AIC:                            -478.4
Df Residuals:                      46   BIC:                            -474.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0072      0.001      6.166      0.0

In [62]:
# Plot SML 
trace  = go.Scatter(x=df_beta.beta, y=df_beta.avg_ret, mode="markers", name='Industries', marker=dict(size=10),text=df_beta.index,
    hovertemplate="<br>industry: %{text}<br>beta: %{x:.2f}<br>avg excess return: %{y:.2%}<extra></extra>")
minval = 0.0
maxval = 2.0
trace_sml = go.Scatter(x= np.linspace(minval,maxval,100), y = theory_slope*np.linspace(minval,maxval,100), mode='lines',name='Theoretical SML',
    hovertemplate=f'Theoretical SML<br>slope: realized market risk premium of {theory_slope:.2%}<extra></extra>')

xvals = np.linspace(minval,maxval,100)
yvals = emp_sml.params['const'] + emp_sml.params['beta']*xvals
trace_emp_sml = go.Scatter(x=xvals, y=yvals , mode='lines',name='Empirical SML', hovertemplate="Empirical SML<extra></extra>")

fig = go.Figure()
fig.add_trace(trace)
fig.add_trace(trace_sml)
fig.add_trace(trace_emp_sml)
fig.update_xaxes(title='Beta',tickformat=".2f", range=[minval,maxval])
fig.update_yaxes(title='Expected Excess Return',tickformat=".1%")
fig.update_layout(title='Theoretical and Empirical Security Market Lines')
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

### Note that the location of the industry relative to the theoretical SML is related to their in-sample alpha.

In [64]:
varname = 'Steel'
y = df[varname]-df['RF']
X = sm.add_constant(df['Mkt-RF'])
mm = sm.OLS(y, X, missing='drop').fit()
print(mm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.587
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     897.9
Date:                Tue, 15 Nov 2022   Prob (F-statistic):          2.28e-123
Time:                        14:31:18   Log-Likelihood:                 981.19
No. Observations:                 633   AIC:                            -1958.
Df Residuals:                     631   BIC:                            -1949.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0031      0.002     -1.517      0.1

## Other portfolios

In [77]:
# 100 portfolios formed on size and book-to-market ratios
d = pdr('100_Portfolios_10x10','famafrench',start=1900)[1]


# Clean-up missings
for c in d.columns:
    d[c] = np.where(d[c]==-99.99, np.nan, d[c])
d = d/100

# Pull and merge market returns
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
df = d.join(ff3[['Mkt-RF','RF']])
df = df.loc['1970-01':].copy()  

In [78]:
df_beta = pd.DataFrame(index=d.columns, columns = ['beta','avg_ret'],dtype=float)
for c in d.columns:
    df_beta.loc[c,:] = params(c)
df_beta

,beta,avg_ret
SMALL LoBM,1.344637,0.001540
ME1 BM2,1.263685,0.005086
ME1 BM3,1.226589,0.007074
ME1 BM4,1.157218,0.007460
ME1 BM5,1.117289,0.008839
...,...,...
ME10 BM6,0.892037,0.007338
ME10 BM7,0.876677,0.003850
ME10 BM8,0.972414,0.007004
ME10 BM9,0.959974,0.005089


In [80]:
theory_slope = df['Mkt-RF'].mean()
print(f'Under CAPM, slope of empirical CAPM should be {theory_slope: .2%} per month.')

Under CAPM, slope of empirical CAPM should be  0.56% per month.


In [81]:
# Empirical SML
y = df_beta.avg_ret
X = sm.add_constant(df_beta.beta)
emp_sml = sm.OLS(y, X, missing='drop').fit()
print(emp_sml.summary())

                            OLS Regression Results                            
Dep. Variable:                avg_ret   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     18.81
Date:                Tue, 15 Nov 2022   Prob (F-statistic):           3.50e-05
Time:                        14:49:44   Log-Likelihood:                 479.34
No. Observations:                 100   AIC:                            -954.7
Df Residuals:                      98   BIC:                            -949.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0143      0.002      9.150      0.0

In [82]:
# Plot SML 
trace  = go.Scatter(x=df_beta.beta, y=df_beta.avg_ret, mode="markers", name='Portfolios', marker=dict(size=10),text=df_beta.index,
    hovertemplate="<br>portfolio: %{text}<br>beta: %{x:.2f}<br>avg excess return: %{y:.2%}<extra></extra>")
minval = 0.0
maxval = 2.0
trace_sml = go.Scatter(x= np.linspace(minval,maxval,100), y = theory_slope*np.linspace(minval,maxval,100), mode='lines',name='Theoretical SML',
    hovertemplate=f'Theoretical SML<br>slope: realized market risk premium of {theory_slope:.2%}<extra></extra>')

xvals = np.linspace(minval,maxval,100)
yvals = emp_sml.params['const'] + emp_sml.params['beta']*xvals
trace_emp_sml = go.Scatter(x=xvals, y=yvals , mode='lines',name='Empirical SML', hovertemplate="Empirical SML<extra></extra>")

fig = go.Figure()
fig.add_trace(trace)
fig.add_trace(trace_sml)
fig.add_trace(trace_emp_sml)
fig.update_xaxes(title='Beta',tickformat=".2f", range=[minval,maxval])
fig.update_yaxes(title='Expected Excess Return',tickformat=".1%")
fig.update_layout(title='Theoretical and Empirical Security Market Lines')
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()